# Imports

In [2]:
#This cell contains all the needed imports

import numpy as np
import matplotlib.pyplot as plt
import random
import pyaudio
import os
import librosa
import pickle
from wav_rw import *
from sklearn.decomposition import PCA
from tools import *
from scipy import signal
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score
import numpy as np
import random
import pyaudio
from wav_rw import wavwrite
from tools import getAudio
import os
import matplotlib.pyplot as plt
import IPython
from scipy.io import wavfile
import noisereduce as nr
import soundfile as sf
from noisereduce.generate_noise import band_limited_noise
import matplotlib.pyplot as plt
import urllib.request
import numpy as np
import io

# Recording audio data set

In [7]:
p = pyaudio.PyAudio()
fs = 16000  # Hertz
duracion = 2
root = "data"

N = int(input("Digite el numero del comandos a grabar (Cuantas clases diferentes): "))
for n in range(N):
    folderClase = input(f"Digite el NOMBRE del comando numero {n+1} a grabar: ")
    if not os.path.exists(root + "/" + folderClase):
        os.makedirs(root + "/" + folderClase)
    name = folderClase.upper()
    N_audios = int(input(f"Cuantos audios va a grabar para {name}?: "))

    folderDestino = root + "/" + folderClase + "/"

    for i in range(N_audios):
        print(f"[INFO] Grabando el audio numero {i+1}: ")
        x = getAudio(p, RATE=fs, RECORD_SECONDS=duracion)
        reduced_noise = nr.reduce_noise(y = x, sr=fs, thresh_n_mult_nonstationary=2,stationary=False)
        x=reduced_noise
        file_name = folderDestino + "/audio" + str(i) + ".wav"
        wavwrite(x, fs, file_name)

print("[INFO] FINALIZADO EL PROCESO, SE HA GENERADO LA BASE DE DATOS")

[INFO] Grabando el audio numero 1: 
[INFO] Escuchando  durante 2 segundos...
[INFO] finalizado!
[INFO] Grabando el audio numero 2: 
[INFO] Escuchando  durante 2 segundos...
[INFO] finalizado!
[INFO] Grabando el audio numero 3: 
[INFO] Escuchando  durante 2 segundos...
[INFO] finalizado!
[INFO] Grabando el audio numero 4: 
[INFO] Escuchando  durante 2 segundos...
[INFO] finalizado!
[INFO] Grabando el audio numero 5: 
[INFO] Escuchando  durante 2 segundos...
[INFO] finalizado!
[INFO] Grabando el audio numero 6: 
[INFO] Escuchando  durante 2 segundos...
[INFO] finalizado!
[INFO] Grabando el audio numero 7: 
[INFO] Escuchando  durante 2 segundos...
[INFO] finalizado!
[INFO] Grabando el audio numero 8: 
[INFO] Escuchando  durante 2 segundos...
[INFO] finalizado!
[INFO] Grabando el audio numero 9: 
[INFO] Escuchando  durante 2 segundos...
[INFO] finalizado!
[INFO] Grabando el audio numero 10: 
[INFO] Escuchando  durante 2 segundos...
[INFO] finalizado!
[INFO] Grabando el audio numero 11: 
[I

# Features Extraction

In [66]:
def read_all_audio_files():
    ## leer todos los archivos
    root = "newData"
    file_names = glob(root + "/**/*.wav", recursive=True)
    clases = glob(root + "/*")
    clases = clases = np.array([clase.split(os.path.sep)[1] for clase in clases])
    print("[INFO] los comandos son:", clases)
    
    x_values = []
    y_values = np.zeros(len(file_names)).astype(int)

    NFFT = 512
    umbral = 0.2
    # leer los audios e iterar

    for i, file_name in enumerate(file_names):
        print(".", end="")
        fs, x = wavread(file_name)
        x = x / np.max(abs(x))
        # filtro 20Hz
        b = signal.firwin(128, 20 / (fs / 2), window="hamming", pass_zero=True)
        s = signal.hilbert(x)
        Eh = np.abs(s)
        Eh = signal.lfilter(b, 1, Eh)
        # tomar solo las partes mayores a 0.2
        x = x[Eh > umbral]
        ## calcular espectro
        M = int(fs * 0.025)
        H = int(fs * 0.010)
        numceps = 13
        mel = melspec_lr(x, fs, numceps, H, M)
        s = np.cov(mel)
        # calcular la triangular superior de la matriz de covarianzas del espectro de mel
        x_values.append(s[np.triu_indices(s.shape[0])])
        for j, clase in enumerate(clases):
            if clase in file_name:
                y_values[i] = j

    pca = PCA().fit(np.vstack(x_values))
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    num_components = np.where(cumulative_variance > 0.95)[0][0] + 1  # 95% de varianza

    print(f"Número de componentes para 95% de varianza explicada: {num_components}")

    pca = PCA(n_components=num_components)
    reduced_features = pca.fit_transform(np.vstack(x_values))

    with open("clases.pkl", "wb") as fh:
        pickle.dump(clases, fh)
    print("[INFO] hecho!!")

    return np.vstack(reduced_features), y_values, pca

# SVM Audio Classifier

In [67]:
x_values,y_values,pca_model = read_all_audio_files()

x_train, x_test, y_train, y_test = train_test_split(x_values,y_values, test_size=0.3)

print("[INFO] entrenando....")

clf = LogisticRegression(solver="lbfgs", max_iter=3000)
clf2 = svm.SVC(kernel='rbf', probability=True)
clf.fit(x_train, y_train)
clf2.fit(x_train, y_train)
print("[INFO] evaluando....")
y_predict = clf.predict(x_test)
y2_predict = clf2.predict(x_test)
acc = accuracy_score(y_test, y_predict)
acc2 = accuracy_score(y_test, y2_predict)

print("[INFO] Porcentaje de prediccion usando LOOGISTICA: ", acc, "\n", "SVM: ", acc2)


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Evaluar el rendimiento del modelo SVM usando validación cruzada
scores = cross_val_score(clf2, x_values, y_values, cv=cv, scoring='accuracy')

# Imprimir los resultados de validación cruzada
print("Precisión en cada pliegue:", scores)
print("Precisión media:", np.mean(scores))

# Definir los parámetros para la búsqueda por grilla
param_grid_svc = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf']
}

# Crear el objeto GridSearchCV
grid_search_svc = GridSearchCV(estimator=svm.SVC(kernel='linear', random_state=42, probability=True), 
                               param_grid=param_grid_svc, 
                               cv=5,  # Número de pliegues en la validación cruzada
                               scoring='accuracy',  # Métrica de evaluación
                               verbose=1)  # Mostrar información durante el entrenamiento

# Ajustar GridSearchCV
grid_search_svc.fit(x_train, y_train)

# Obtener el mejor modelo
best_svc = grid_search_svc.best_estimator_

# Hacer predicciones con el mejor modelo
pred_svc = best_svc.predict(x_test)

# Evaluar el modelo
accuracy_svc = accuracy_score(y_test, pred_svc)

print("ACCURACY MEJOR SVM", accuracy_svc)


# guardar para posterior uso
print("[INFO] guardando el modelo")
with open("clasificador.pkl", "wb") as fh:
    pickle.dump(best_svc, fh)

with open("pca_model.pkl", "wb") as fh:
    pickle.dump(pca_model, fh)
print("[INFO] hecho!!")

[INFO] los comandos son: ['down' 'left' 'right' 'up']
......................

c:\Users\sebas\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


..................................................................................................................................................................................

c:\Users\sebas\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=512 is too large for input signal of length=465
  warnings.warn(
c:\Users\sebas\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=512 is too large for input signal of length=324
  warnings.warn(


Número de componentes para 95% de varianza explicada: 98
[INFO] hecho!!
[INFO] entrenando....
[INFO] evaluando....
[INFO] Porcentaje de prediccion usando LOOGISTICA:  0.9 
 SVM:  0.9166666666666666
Precisión en cada pliegue: [0.925 0.95  0.95  0.975 0.925]
Precisión media: 0.9450000000000001
Fitting 5 folds for each of 8 candidates, totalling 40 fits
ACCURACY MEJOR SVM 0.9166666666666666
[INFO] guardando el modelo
[INFO] hecho!!


# Classes Predictions

In [94]:
import numpy as np
import random
import pyaudio
from wav_rw import wavwrite
from tools import getAudio
import os
import matplotlib.pyplot as plt


import IPython
from scipy.io import wavfile
import noisereduce as nr
import soundfile as sf
from noisereduce.generate_noise import band_limited_noise
import matplotlib.pyplot as plt
import urllib.request
import numpy as np
import io

# 1. Crear objeto de pyAudio
p = pyaudio.PyAudio()
fs = 16000  # Hertz
duracion = 2  # cuantos segundos por audio?

##2. cargar el modelo y las etiquetas
fh = open("clasificador.pkl", "rb")
clf = pickle.load(fh)
fh.close()

# etiquetas
fh = open("clases.pkl", "rb")
clases = pickle.load(fh)
fh.close()


# pca_model
fh = open("pca_model.pkl", "rb")
pca_model = pickle.load(fh)
fh.close()

##3. capturar audio usando el microfono y hacer todo el proceso hasta calcular las caracteristicas
# filtro 20hz
b = signal.firwin(128, 20 / (fs / 2), window="hamming", pass_zero=True)
x = getAudio(p, RATE=fs, RECORD_SECONDS=2)
reduced_noise = nr.reduce_noise(y = x, sr=fs, thresh_n_mult_nonstationary=2,stationary=False)
x=reduced_noise
print(max(reduced_noise))
# Crear un stream de salida para la reproducción
stream = p.open(format=pyaudio.paFloat32,
                channels=1,
                rate=fs,
                output=True)

# Reproducir el audio
stream.write(x.astype(np.float32).tobytes())

# Cerrar el stream
stream.stop_stream()
stream.close()
p.terminate()

x = x / np.max(abs(x))
s = signal.hilbert(x)
Eh = np.abs(s)
Eh = signal.lfilter(b, 1, Eh)
# tomar solo las partes mayores a 0.2, xs es la senal segmentada
xs = x[Eh > 0.2]
## calcular espectro de mel
M = int(fs * 0.025)
H = int(fs * 0.010)
numceps = 13
mel = melspec_lr(xs, fs, numceps, H, M)
s = np.cov(mel)
# calcular la triangular superior de la matriz de covarianzas del espectro de mel
feats = s[np.triu_indices(s.shape[0])]

feats_pca = pca_model.transform(feats.reshape(1, -1))
# 4. realizar prediccion usando el modelo
prediction = clf.predict(feats_pca)
print(f"[INFO] comando reconocido: {clases[prediction][0]}")
probability = clf.predict_proba(feats_pca.reshape(1, -1))
print(probability)

[INFO] Escuchando  durante 2 segundos...
[INFO] finalizado!
0.7068822
[INFO] comando reconocido: right
[[0.07910604 0.01748391 0.5630418  0.34036825]]


c:\Users\sebas\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)
